In [1]:
### M3 Week 2 Group 8: Daniel Azemar, María Gil, Richard Segovia

In [ ]:
import cv2
import numpy as np
import pickle
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier

train_images_filenames = pickle.load(open('./train_images_filenames.dat', 'rb'))
test_images_filenames = pickle.load(open('./test_images_filenames.dat', 'rb'))
train_labels = pickle.load(open('./train_labels.dat', 'rb'))
test_labels = pickle.load(open('./test_labels.dat', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: './train_images_filenames.dat'

In [7]:
for num_features in [300, 400, 500, 600, 700, 800, 900, 1000]:
    SIFTdetector = cv2.xfeatures2d.SIFT_create(nfeatures=num_features)
    
    Train_descriptors = []
    Train_label_per_descriptor = []

    for filename,labels in zip(train_images_filenames,train_labels):
        ima=cv2.imread(filename)
        gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
        kpt,des=SIFTdetector.detectAndCompute(gray,None)
        Train_descriptors.append(des)
        Train_label_per_descriptor.append(labels)

    D=np.vstack(Train_descriptors)
    
    k = 128
    codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
    codebook.fit(D)
    
    visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
    for i in range(len(Train_descriptors)):
        words=codebook.predict(Train_descriptors[i])
        visual_words[i,:]=np.bincount(words,minlength=k)
        
    knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1,metric='euclidean')
    knn.fit(visual_words, train_labels) 
    
    visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
    for i in range(len(test_images_filenames)):
        filename=test_images_filenames[i]
        ima=cv2.imread(filename)
        gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
        kpt,des=SIFTdetector.detectAndCompute(gray,None)
        words=codebook.predict(des)
        visual_words_test[i,:]=np.bincount(words,minlength=k)
    accuracy = 100*knn.score(visual_words_test, test_labels)
    print(num_features, accuracy)

300 54.770755885997524
400 57.125154894671624
500 56.877323420074354
600 60.718711276332094
700 57.62081784386617
800 60.594795539033456
900 57.74473358116481
1000 59.10780669144982


In [5]:
SIFTdetector = cv2.xfeatures2d.SIFT_create(nfeatures=600)

Train_descriptors = []
Train_label_per_descriptor = []

for filename,labels in zip(train_images_filenames,train_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    kpt,des=SIFTdetector.detectAndCompute(gray,None)
    Train_descriptors.append(des)
    Train_label_per_descriptor.append(labels)

D=np.vstack(Train_descriptors)

In [6]:
for k in [64, 128, 256 , 512, 1024, 2048]:

    #k = 128
    codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
    codebook.fit(D)
    
    visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
    for i in range(len(Train_descriptors)):
        words=codebook.predict(Train_descriptors[i])
        visual_words[i,:]=np.bincount(words,minlength=k)
        
    knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1,metric='euclidean')
    knn.fit(visual_words, train_labels) 
    
    visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
    for i in range(len(test_images_filenames)):
        filename=test_images_filenames[i].replace("../../","")
        ima=cv2.imread(filename)
        gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
        kpt,des=SIFTdetector.detectAndCompute(gray,None)
        words=codebook.predict(des)
        visual_words_test[i,:]=np.bincount(words,minlength=k)
    accuracy = 100*knn.score(visual_words_test, test_labels)
    print(k, accuracy)

error: OpenCV(3.4.2) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'


In [ ]:
k = 128
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
for i in range(len(Train_descriptors)):
    words=codebook.predict(Train_descriptors[i])
    visual_words[i,:]=np.bincount(words,minlength=k)


In [ ]:
for num_neighbors in [3, 4, 5, 6, 7, 8]: 
    knn = KNeighborsClassifier(n_neighbors=num_neighbors,n_jobs=-1,metric='euclidean')
    knn.fit(visual_words, train_labels) 
    
    visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
    for i in range(len(test_images_filenames)):
        filename=test_images_filenames[i].replace("../../","")
        ima=cv2.imread(filename)
        gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
        kpt,des=SIFTdetector.detectAndCompute(gray,None)
        words=codebook.predict(des)
        visual_words_test[i,:]=np.bincount(words,minlength=k)
    accuracy = 100*knn.score(visual_words_test, test_labels)
    print(num_neighbors, accuracy)

In [14]:
for distance in ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan']:
    knn = KNeighborsClassifier(n_neighbors=6,n_jobs=-1,metric=distance)
    knn.fit(visual_words, train_labels) 

    visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
    for i in range(len(test_images_filenames)):
        filename=test_images_filenames[i].replace("../../","")
        ima=cv2.imread(filename)
        gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
        kpt,des=SIFTdetector.detectAndCompute(gray,None)
        words=codebook.predict(des)
        visual_words_test[i,:]=np.bincount(words,minlength=k)
    accuracy = 100*knn.score(visual_words_test, test_labels)
    print(distance, accuracy)

cityblock 58.98389095415118
cosine 60.84262701363073
euclidean 60.84262701363073
l1 58.98389095415118
l2 60.84262701363073
manhattan 58.98389095415118
